In [25]:
pip install paho-mqtt

In [26]:
pip install python-barcode yagmail

In [27]:
!apt-get update && apt-get install -y libzbar0

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [28]:
pip install pyzbar opencv-python

In [29]:
from pyzbar import pyzbar
import paho.mqtt.client as mqtt
import cv2

In [30]:
#Informacion imagen
class InfoImg:
    def __init__(self, tipo, datos):
        self.tipo = tipo
        self.datos = datos

In [35]:
def decode(image):
    # Decodes all barcodes from an image
    decoded_objects = pyzbar.decode(image)

    if not decoded_objects:
        print("No barcodes detected.")
        return None  # Retorna None si no hay códigos de barras

    # Procesar solo el primer código de barras detectado
    obj = decoded_objects[0]
    print("Detected barcode:", obj)
    image = draw_barcode(obj, image)

    # Decodificar el tipo y los datos
    tipo = obj.type
    datos = obj.data.decode("utf-8")  # Decodificar los datos a texto
    print("Type:", tipo)
    print("Data:", datos)
    print()

    # Retornar una instancia de InfoImg
    return InfoImg(tipo, datos)

def draw_barcode(decoded, image):
    # n_points = len(decoded.polygon)
    # for i in range(n_points):
    #     image = cv2.line(image, decoded.polygon[i], decoded.polygon[(i+1) % n_points], color=(0, 255, 0), thickness=5)
    # uncomment above and comment below if you want to draw a polygon and not a rectangle
    image = cv2.rectangle(image, (decoded.rect.left, decoded.rect.top),
                            (decoded.rect.left + decoded.rect.width, decoded.rect.top + decoded.rect.height),
                            color=(0, 255, 0),
                            thickness=5)
    return image

# def publicador(infoImg, numeroGrupo):
#     client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)  # Actualizar API
#     client.connect("test.mosquitto.org", 1883, 60)

#     if str(infoImg.datos)[0] == str(numeroGrupo):
#         siguiente_grupo = str(infoImg.datos)[1]
#         nuevo_dato = int(infoImg.datos) + 10
#         client.publish(f"PATRONES2025/G{siguiente_grupo}", nuevo_dato)
#         print(f"Enviando {nuevo_dato} al grupo {siguiente_grupo}")
#         client.disconnect()  # Desconectar después de enviar
#         return

def publicador(infoImg, numeroGrupo):
    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)  # Usar API V2
    client.connect("test.mosquitto.org", 1883, 60)

    # Caso 1: Grupo inicial (3)
    if str(infoImg.datos)[0] == str(numeroGrupo):
        siguiente_grupo = str(infoImg.datos)[1]  # Obtener el siguiente dígito (1)
        nuevo_dato = int(infoImg.datos) + 10  # 312546789760 → 312546789770
        client.publish(f"PATRONES2025/all", "Del grupo 3 al grupo "+ siguiente_grupo)
        client.publish(f"PATRONES2025/G{siguiente_grupo}", nuevo_dato)
        print(f"Enviando {nuevo_dato} al grupo {siguiente_grupo}")
        client.disconnect()

        # Llamada recursiva para el siguiente grupo (1)
        publicador(infoImg, int(siguiente_grupo))  # Ahora procesa el grupo 1

    else:
        # Caso 2: Grupo intermedio (1)
        def on_message(client, userdata, message):
            print(f"Mensaje recibido: {message.payload.decode()}")
            nuevo_dato = int(message.payload.decode()) + 10
            datos_str = str(infoImg.datos)

            if str(numeroGrupo) not in datos_str:
                print("Grupo no está en los datos")
                client.disconnect()
                return

            index_actual = datos_str.index(str(numeroGrupo))

            #Caso 3: En caso de recibir el mensaje para G1, envia el correo
            if index_actual == 4:
              send_email(str(nuevo_dato))
              client.publish(f"PATRONES2025/all", f"Enviado al correo")
              return

            if index_actual + 1 >= len(datos_str):
                print("Fin de la secuencia")
                client.disconnect()
                return

            siguiente_grupo = datos_str[index_actual + 1]
            client.publish(f"PATRONES2025/G{siguiente_grupo}", nuevo_dato)
            # client.publish(f"PATRONES2025/#", "GRUPO " + numeroGrupo + " a GRUPO" + siguiente_grupo)
            print(f"Publicado a G{siguiente_grupo}: {nuevo_dato}")
            client.disconnect()

        client.on_message = on_message
        client.subscribe(f"PATRONES2025/G{numeroGrupo}")
        print(f"Escuchando en PATRONES2025/G{numeroGrupo}...")
        client.loop_forever()

from barcode import EAN13
from barcode.writer import ImageWriter
import yagmail

def send_email(payload: str):
    # try:
        # Validar que el código tenga 12 o 13 dígitos numéricos
        if not (len(payload) in (12, 13) and payload.isdigit()):
            raise ValueError("El código debe contener 12 o 13 dígitos numéricos.")

        # Crear el código EAN-13 (el dígito de control se calcula automáticamente si son 12 dígitos)
        ean = EAN13(payload, writer=ImageWriter())

        # 2. Verifica que `ean` no sea None
        if ean is None:
            raise ValueError("Formato de código de barras no válido. ¿Está instalado python-barcode?")

        # 3. Genera el código de barras
        # bar = ean(payload, writer=ImageWriter())
        filename = ean.save("barcode")  # Guarda como "barcode.png"

        # 4. Envía el email
        user = "sierraincollege@gmail.com"
        to_email = "kamisops@gmail.com"
        subject = "Codigo Taller G1"

        gmail = yagmail.SMTP(user, password="bsok bfqe atlu omjy")
        gmail.send(to_email, subject, [filename])  # Añade extensión .png
        print(f"Email sent to {to_email}")

    # except Exception as e:
    #     print(f"Error al generar el código EAN-13: {e}")
    #     return None

In [ ]:
if __name__ == "__main__":
    from glob import glob

    barcodes = glob("Caso3.png")

    for barcode_file in barcodes:
        # Cargar la imagen con OpenCV
        img = cv2.imread(barcode_file)

        # Decodificar el primer código de barras detectado
        decoded_info = decode(img)

        if decoded_info is not None:
            print(f"Tipo: {decoded_info.tipo}, Datos: {decoded_info.datos}")
            # Intentar convertir los datos a un número
            try:
                datos = int(decoded_info.datos)  # Convertir los datos a un número entero si es posible
                publicador(decoded_info, 1)

            except ValueError:
                print("No contiene datos, terminado programa")
                pass  # Si no es un número, mantener los datos como texto
        else:
            print("No se detectaron códigos de barras.")



Detected barcode: Decoded(data=b'3675163619849', type='EAN13', rect=Rect(left=76, top=11, width=371, height=177), polygon=[Point(x=76, y=11), Point(x=76, y=187), Point(x=267, y=188), Point(x=447, y=188), Point(x=447, y=12)], quality=355, orientation='UP')
Type: EAN13
Data: 3675163619849

Tipo: EAN13, Datos: 3675163619849
Escuchando en PATRONES2025/G1...
Mensaje recibido: 3675163619889
Email sent to kamisops@gmail.com
